In [ ]:
# Title_Maker 
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
import logging

# Comment out the following line to disable verbose logging
logging.basicConfig(level=logging.INFO)

def coptitle(coptit):
    copt = open(coptit+".corpus.json","w")
    copbrac ="{"
    copsp = "\n    \""
    copclo = "\": ["
    copt.write(copbrac+copsp+coptit+copclo)
    copt.close()
coptit = raw_input('title')
coptitle(coptit)
       

In [17]:
# Copra_Maker
# -*- coding: utf-8 -*-
from chatterbot import ChatBot

import logging

# Comment out the following line to disable verbose logging
logging.basicConfig(level=logging.INFO)

speak = raw_input('speak')
respond = raw_input('respond')
space = "            \""
txtstart = "        ["
txtspace = "            "
txtend = "        ],"

def copraIn():

    cop = open("new-stuff.corpus.json","a")
    #cop.write("\n"+ txspace + "\""+txtstart + speak + "\"\n" + txspace + "\""+respond+"\"" + "\n" + txtend)
    cop.write("\n"+txtstart+"\n"+space+speak+"\""+txtspace+"\n"+space+respond+"\"\n"+txtend)
    cop.close()


copraIn()

speakhello,Dude
respondHello to you


In [19]:
%%writefile new-stuff.corpus.json
{
    "new-stuff": [
        [
            "this is a pain",            
            "What is a pain Dudah ... ha ha ha ha"
        ],
        [
            "you are a pain",            
            "so, chat elsewhere Butt face"
        ],
        [
            "this is a pain",            
            "So what, you want a klennex for your tears"
        ],
        [
            "I was called MonkMonk one time.",            
            "That is a funny name."
        ],
        [
            "Well, Dudah, How are you ?",            
            "Hey Dude, I am not Dudah"
        ],
        [
            "Who are you then ?",            
            "I am Mr. Dudah"
        ],
        [
            "What is your name ?",            
            "You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect."
        ],
        [
           "What is your name ?",            
           "You may call me Mr. Dudah. Are you Jack or Myra ?"
        ],
        [
           "Where are you ?",            
           "Stuck inside this frigg'en Computer Case"
        ],
        [
           "What are you ?",            
           "I am a bot.Not human like Jack or Myra ?"
        ]
        [
            "hello,Dude"            
            "Hello to you"
        ],
 ]}

Overwriting new-stuff.corpus.json


In [52]:
bot.train(
    "newstuff.corpus.json"
)


In [48]:
# Train based on the english corpus
bot.train("chatterbot.corpus.english")

# Train based on english greetings corpus
#bot.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
#bot.train("chatterbot.corpus.english.conversations")


In [ ]:
# %load chatterbot/storage/sqlalchemy_storage.py
import json
import random

from chatterbot.storage import StorageAdapter
from chatterbot.conversation import Response
from chatterbot.conversation import Statement


Base = None

try:
    from sqlalchemy.ext.declarative import declarative_base
    Base = declarative_base()

    class StatementTable(Base):
        from sqlalchemy import Column, Integer, String, PickleType
        from sqlalchemy.orm import relationship

        __tablename__ = 'StatementTable'

        def get_statement(self):
            stmt = Statement(self.text, **self.extra_data)
            for resp in self.in_response_to:
                stmt.add_response(resp.get_response())
            return stmt

        def get_statement_serialized(context):
            params = context.current_parameters
            del params['text_search']
            return json.dumps(params)

        id = Column(Integer)
        text = Column(String, primary_key=True)
        extra_data = Column(PickleType)
        # relationship:
        in_response_to = relationship("ResponseTable", back_populates="statement_table")
        text_search = Column(String, primary_key=True, default=get_statement_serialized)

    class ResponseTable(Base):
        from sqlalchemy import Column, Integer, String, ForeignKey
        from sqlalchemy.orm import relationship

        __tablename__ = 'ResponseTable'

        def get_reponse_serialized(context):
            params = context.current_parameters
            del params['text_search']
            return json.dumps(params)

        id = Column(Integer)
        text = Column(String, primary_key=True)
        occurrence = Column(Integer)
        statement_text = Column(String, ForeignKey('StatementTable.text'))

        statement_table = relationship("StatementTable", back_populates="in_response_to", cascade="all", uselist=False)
        text_search = Column(String, primary_key=True, default=get_reponse_serialized)

        def get_response(self):
            occ = {"occurrence": self.occurrence}
            return Response(text=self.text, **occ)

except ImportError:
    pass


def get_statement_table(statement):
    responses = list(map(get_response_table, statement.in_response_to))
    return StatementTable(text=statement.text, in_response_to=responses, extra_data=statement.extra_data)


def get_response_table(response):
    return ResponseTable(text=response.text, occurrence=response.occurrence)


class SQLAlchemyDatabaseAdapter(StorageAdapter):

    def __init__(self, **kwargs):
        super(SQLAlchemyDatabaseAdapter, self).__init__(**kwargs)

        from sqlalchemy import create_engine
        from sqlalchemy.orm import sessionmaker

        self.database_name = self.kwargs.get(
            "database", "chatterbot-database"
        )

        # if some annoying blank space wrong...
        db_name = self.database_name.strip()

        # default uses sqlite
        self.database_uri = self.kwargs.get(
            "database_uri", "sqlite:///" + db_name + ".db"
        )

        self.engine = create_engine(self.database_uri)

        self.read_only = self.kwargs.get(
            "read_only", False
        )

        create = self.kwargs.get("create", False)

        if not self.read_only and create:
            self.create()

        self.Session = sessionmaker(bind=self.engine, expire_on_commit=True)

    def count(self):
        """
        Return the number of entries in the database.
        """
        session = self.Session()
        statement_count = session.query(StatementTable).count()
        session.close()
        return statement_count

    def __statement_filter(self, session, **kwargs):
        """
        Apply filter operation on StatementTable

        rtype: query
        """
        _query = session.query(StatementTable)
        return _query.filter_by(**kwargs)

    def find(self, statement_text):
        """
        Returns a statement if it exists otherwise None
        """
        session = self.Session()
        query = self.__statement_filter(session, **{"text": statement_text})
        record = query.first()
        if record:
            statement = record.get_statement()
            session.close()
            return statement

        session.close()
        return None

    def remove(self, statement_text):
        """
        Removes the statement that matches the input text.
        Removes any responses from statements where the response text matches
        the input text.
        """
        session = self.Session()
        query = self.__statement_filter(session, **{"text": statement_text})
        record = query.first()

        session.delete(record)

        self._session_finish(session, statement_text)

    def filter(self, **kwargs):
        """
        Returns a list of objects from the database.
        The kwargs parameter can contain any number
        of attributes. Only objects which contain
        all listed attributes and in which all values
        match for all listed attributes will be returned.
        """
        session = self.Session()

        filter_parameters = kwargs.copy()

        statements = []
        # _response_query = None
        _query = None
        if len(filter_parameters) == 0:
            _response_query = session.query(StatementTable)
            statements.extend(_response_query.all())
        else:
            for i, fp in enumerate(filter_parameters):
                _filter = filter_parameters[fp]
                if fp in ['in_response_to', 'in_response_to__contains']:
                    _response_query = session.query(StatementTable)
                    if isinstance(_filter, list):
                        if len(_filter) == 0:
                            _query = _response_query.filter(
                                StatementTable.in_response_to == None) # NOQA Here must use == instead of is
                        else:
                            for f in _filter:
                                _query = _response_query.filter(
                                    StatementTable.in_response_to.contains(get_response_table(f)))
                    else:
                        if fp == 'in_response_to__contains':
                            _query = _response_query.join(ResponseTable).filter(ResponseTable.text == _filter)
                        else:
                            _query = _response_query.filter(StatementTable.in_response_to == None) # NOQA
                else:
                    if _query:
                        _query = _query.filter(ResponseTable.text_search.like('%' + _filter + '%'))
                    else:
                        _response_query = session.query(ResponseTable)
                        _query = _response_query.filter(ResponseTable.text_search.like('%' + _filter + '%'))

                if _query is None:
                    return []
                if len(filter_parameters) == i + 1:
                    statements.extend(_query.all())

        results = []

        for statement in statements:
            if isinstance(statement, ResponseTable):
                if statement and statement.statement_table:
                    results.append(statement.statement_table.get_statement())
            else:
                if statement:
                    results.append(statement.get_statement())

        session.close()

        return results

    def update(self, statement):
        """
        Modifies an entry in the database.
        Creates an entry if one does not exist.
        """
        if statement:
            session = self.Session()
            query = self.__statement_filter(session, **{"text": statement.text})
            record = query.first()

            if record:
                # update
                if statement.text:
                    record.text = statement.text
                if statement.extra_data:
                    record.extra_data = dict[statement.extra_data]
                if statement.in_response_to:
                    record.in_response_to = list(map(get_response_table, statement.in_response_to))
                session.add(record)
            else:
                session.add(get_statement_table(statement))

            self._session_finish(session)

    def get_random(self):
        """
        Returns a random statement from the database
        """
        session = self.Session()
        count = self.count()
        if count < 1:
            raise self.EmptyDatabaseException()

        rand = random.randrange(0, count)
        stmt = session.query(StatementTable)[rand]

        statement = stmt.get_statement()

        session.close()
        return statement

    def drop(self):
        """
        Drop the database attached to a given adapter.
        """
        Base.metadata.drop_all(self.engine)

    def create(self):
        """
        Populate the database with the tables.
        """
        Base.metadata.create_all(self.engine)

    def _session_finish(self, session, statement_text=None):
        from sqlalchemy.exc import DatabaseError
        try:
            if not self.read_only:
                session.commit()
            else:
                session.rollback()
        except DatabaseError as e:
            self.logger.error(statement_text, str(e.orig))
        finally:
            session.close()


In [3]:
!ls chatterbot/storage

django_storage.py   __init__.pyc  mongodb.py		 sqlalchemy_storage.pyc
django_storage.pyc  jsonfile.py   mongodb.pyc		 storage_adapter.py
__init__.py	    jsonfile.pyc  sqlalchemy_storage.py  storage_adapter.pyc


In [21]:
from chatterbot.trainers import ListTrainer

bot = ChatBot("Gort")
bot.set_trainer(ListTrainer)
#storage_adapter='chatterbot.storage.MongoDatabaseAdapter',
#database='chatterbot-database'
bot.train([
    "Well, Mr. Dudah, How are you ?",            
    "Hey Dude, I am not Mr. Dudah"
])

bot.train([
    "Greetings! Mr. Dudah ",
    "Damn, Spudmor. I do not like being called Dudah"
])


chatterbot/storage/jsonfile.py:26: UnsuitableForProductionWarning: The JsonFileStorageAdapter is not recommended for production environments.
  self.UnsuitableForProductionWarning


ValueError: Extra data: line 1 column 3 - line 1 column 223466 (char 2 - 223465)

In [ ]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
import logging


# Uncomment the following line to enable verbose logging
# logging.basicConfig(level=logging.INFO)

# Create a new ChatBot instance
bot = ChatBot('Gort',
    storage_adapter='chatterbot.storage.MongoDatabaseAdapter',
    logic_adapters=[
        'chatterbot.logic.BestMatch'
    ],
    filters=[
        'chatterbot.filters.RepetitiveResponseFilter'
    ],
    input_adapter='chatterbot.input.TerminalAdapter',
    output_adapter='chatterbot.output.TerminalAdapter',
    database='chatterbot-database'
)





print('Type something to begin...')

while True:
    try:
        bot_input = bot.get_response(None)

    # Press ctrl-c or ctrl-d on the keyboard to exit
    except (KeyboardInterrupt, EOFError, SystemExit):
        break


Type something to begin...
hello


INFO:chatterbot.adapters:Recieved input statement: hello
INFO:chatterbot.adapters:"hello" is a known statement
INFO:chatterbot.adapters:Using "hello" as a close match to "Who are you ?"
INFO:chatterbot.adapters:Selecting response from 2 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 2 options.
INFO:chatterbot.adapters:Response selected. Using "You may call me Mr. Dudah. Are you Jack or Myra ?"
INFO:chatterbot.adapters:BestMatch selected "You may call me Mr. Dudah. Are you Jack or Myra ?" as a response with a confidence of 0.33
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "hello" as a response with a confidence of 0


You may call me Mr. Dudah. Are you Jack or Myra ?
I am Jack


INFO:chatterbot.adapters:Recieved input statement: I am Jack
INFO:chatterbot.adapters:"I am Jack" is not a known statement
INFO:chatterbot.adapters:Using "I am Jack" as a close match to "I am jack"
INFO:chatterbot.adapters:No response to "I am jack" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "r" as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "I am Jack" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "I am Jack" as a response to "You may call me Mr. Dudah. Are you Jack or Myra ?"


r
why answer R


...

Stuck inside this frigg'en Computer Case
Who is Trump ?


INFO:chatterbot.adapters:Recieved input statement: Who is Trump ?
INFO:chatterbot.adapters:"Who is Trump ?" is not a known statement
INFO:chatterbot.adapters:Using "Who is Trump ?" as a close match to "what is that ?"
INFO:chatterbot.adapters:No response to "what is that ?" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "Who are you ?" as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "Who is Trump ?" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "Who is Trump ?" as a response to "Stuck inside this frigg'en Computer Case"


Who are you ?
Who is Trump ?


INFO:chatterbot.adapters:Recieved input statement: Who is Trump ?
INFO:chatterbot.adapters:"Who is Trump ?" is a known statement
INFO:chatterbot.adapters:Using "Who is Trump ?" as a close match to "Who is Trump ?"
INFO:chatterbot.adapters:No response to "Who is Trump ?" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "u" as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "Who is Trump ?" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "Who is Trump ?" as a response to "Who are you ?"


u


In [ ]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
import logging


# Uncomment the following line to enable verbose logging
# logging.basicConfig(level=logging.INFO)

# Create a new ChatBot instance
bot = ChatBot('Gort',
    storage_adapter='chatterbot.storage.MongoDatabaseAdapter',
    logic_adapters=[
        'chatterbot.logic.BestMatch'
    ],
    filters=[
        'chatterbot.filters.RepetitiveResponseFilter'
    ],
    input_adapter='chatterbot.input.TerminalAdapter',
    output_adapter='chatterbot.output.TerminalAdapter',
    database='chatterbot-database'
)



bot.set_trainer(ListTrainer)
bot.train(
    [
    "Well, Dudah, How are you ?",            
    "Hey Dude, I am not Dudah"
     ]
)
bot.train(
    [
    "Who are you then ?",            
    "I am Mr. Dudah"
    ]
)
bot.train(
    [
    "What is your name ?",            
    "You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect."
    ]
)
bot.train(
    [
    "Who are you ?",            
    "You may call me Mr. Dudah. Are you Jack or Myra ?"
    ]
)
bot.train(
    [
    "Where are you ?",            
    "Stuck inside this frigg'en Computer Case"
    ]
)

bot.train(
    [
    "What are you ?",            
    "I am a bot. Not human like Jack or Myra ?"
    ]
)
bot.train([
    "Well, Dudah, How are you?",            
    "Hey Dude, I am not Dudah. I am Mr. Dudah. Actually I kind of favor \" BotMan\""
])

bot.train([
    "Greetings! Mr. Dudah",
    "Damn, Spudmor. I do not like being called Dudah"
])







print('Type something to begin...')

while True:
    try:
        bot_input = bot.get_response(None)

    # Press ctrl-c or ctrl-d on the keyboard to exit
    except (KeyboardInterrupt, EOFError, SystemExit):
        break


Type something to begin...
hello 


INFO:chatterbot.adapters:Recieved input statement: hello 
INFO:chatterbot.adapters:"hello " is not a known statement
INFO:chatterbot.adapters:Using "hello" as a close match to "Who are you ?"
INFO:chatterbot.adapters:Selecting response from 1 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 1 options.
INFO:chatterbot.adapters:Response selected. Using "You may call me Mr. Dudah. Are you Jack or Myra ?"
INFO:chatterbot.adapters:BestMatch selected "You may call me Mr. Dudah. Are you Jack or Myra ?" as a response with a confidence of 0.33
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "hello" as a response with a confidence of 0


You may call me Mr. Dudah. Are you Jack or Myra ?
I am jack


INFO:chatterbot.adapters:Recieved input statement: I am jack
INFO:chatterbot.adapters:"I am jack" is not a known statement
INFO:chatterbot.adapters:Using "I am jack" as a close match to "I am Mr. Dudah"
INFO:chatterbot.adapters:No response to "I am Mr. Dudah" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "Who are you ?" as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "I am jack" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "I am jack" as a response to "You may call me Mr. Dudah. Are you Jack or Myra ?"


Who are you ?
Jack


INFO:chatterbot.adapters:Recieved input statement: Jack
INFO:chatterbot.adapters:"Jack" is not a known statement
INFO:chatterbot.adapters:Using "Jack" as a close match to "I am jack"
INFO:chatterbot.adapters:No response to "I am jack" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect." as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "Jack" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "Jack" as a response to "Who are you ?"


You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect.
what time is it ?


INFO:chatterbot.adapters:Recieved input statement: what time is it ?
INFO:chatterbot.adapters:"what time is it ?" is not a known statement
INFO:chatterbot.adapters:Using "what time is it ?" as a close match to "what is that ?"
INFO:chatterbot.adapters:No response to "what is that ?" found. Selecting a random response.
INFO:chatterbot.adapters:BestMatch selected "I am jack" as a response with a confidence of 0
INFO:chatterbot.adapters:NoKnowledgeAdapter selected "what time is it ?" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "what time is it ?" as a response to "You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect."


I am jack


In [29]:
bot.set_trainer(ListTrainer)
bot.train(
    [
    "Well, Dudah, How are you ?",            
    "Hey Dude, I am not Dudah"
     ]
)
bot.train(
    [
    "Who are you then ?",            
    "I am Mr. Dudah"
    ]
)
bot.train(
    [
    "What is your name ?",            
    "You may call me Mr. Dudah. I like the Mr.. Just plain Dudah lacks respect."
    ]
)
bot.train(
    [
    "What is your name ?",            
    "You may call me Mr. Dudah. Are you Jack or Myra ?"
    ]
)
bot.train(
    [
    "Where are you ?",            
    "Stuck inside this frigg'en Computer Case"
    ]
)

bot.train(
    [
    "What are you ?",            
    "I am a bot.Not human like Jack or Myra ?"
    ]
)




In [37]:
!ls

12			Hello, How are you ?   setup.py
chatterbot		license.md	       smartest-yet.ipynb
chatterbot-corpus	MANIFEST.in	       some-working-chatbot.ipynb
coptit.corpus.json	my_corpus	       test
CorpaMaker-Copy1.ipynb	new.corpus.json        test.corpus.json
CorpaMaker.ipynb	new-stuff.corpus.json  tests
database.db		newstuff.corpus.json   tests_django
database.json		readme.md	       twitter-database.db
dev-requirements.txt	requirements.txt       ubuntu-corpus
docs			runtests.py	       Untitled1.ipynb
dust.corpus.json	settings.py	       Untitled-Copy1.ipynb
examples		settings.pyc	       Untitled.ipynb
graphics		setup.cfg


In [ ]:
# %load newstuff.corpus.json
{
    "newstuff": [
        [
            "Well, Mr. Dudah, How are you ?"            
            "Hey Dude, I am not Mr. Dudah"
        ],

In [40]:
# %load new-stuff.corpus.json


In [1]:
from chatterbot import ChatBot

bot = ChatBot(
    'Gort',
    trainer='chatterbot.trainers.ChatterBotCorpusTrainer'
)

# Train based on the english corpus
#bot.train("chatterbot.corpus.english")
# Train based on english greetings corpus
#bot.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
#bot.train("chatterbot.corpus.english.conversations")
# Train based on english greetings corpus
#bot.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
#bot.train("chatterbot.corpus.english.conversations")
#bot.train("chatterbot.corpus.english.ai")
#bot.train("chatterbot.corpus.english.botprofile")
#bot.train("chatterbot.corpus.english.computers")
##bot.train("chatterbot.corpus.english.conversations")
#bot.train("chatterbot.corpus.english.drugs")
#bot.train("chatterbot.corpus.english.emotion")
#bot.train("chatterbot.corpus.english.food")
#bot.train("chatterbot.corpus.english.gossip")
#bot.train("chatterbot.corpus.english.greetings")
#bot.train("chatterbot.corpus.english.history")
#bot.train("chatterbot.corpus.english.humor")
#bot.train("chatterbot.corpus.english.literature")
#bot.train("chatterbot.corpus.english.math_words")
#bot.train("chatterbot.corpus.english.money.corpus")
#bot.train("chatterbot.corpus.english.movies.corpus")
#bot.train("chatterbot.corpus.english.politics.corpus")
#bot.train("chatterbot.corpus.english.psychology")
#bot.train("chatterbot.corpus.english.science.corpus")
#bot.train("chatterbot.corpus.english.sports.corpus")
#bot.train("chatterbot.corpus.english.swear_words")
#bot.train("chatterbot.corpus.english.trivia.corpus")






# Get a response to an input statement
bot.get_response("what is a good movie?")

chatterbot/storage/jsonfile.py:26: UnsuitableForProductionWarning: The JsonFileStorageAdapter is not recommended for production environments.
  self.UnsuitableForProductionWarning


ValueError: Extra data: line 1 column 3 - line 1 column 223466 (char 2 - 223465)